# Lab - Customer Analysis Round 7

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import time 
from datetime import date


# Try to improve the linear regression model

In [2]:
numerical = pd.read_csv('datalab2_numerical.csv')
numerical_x=pd.read_csv('datalab2_numerical_x.csv')
categorical_x=pd.read_csv('datalab2_categorical_x.csv')
numerical_x = numerical_x.drop(['Unnamed: 0'],axis=1)
categorical = pd.read_csv('datalab2_categorical.csv')
categorical_x = categorical_x.drop(['Unnamed: 0'],axis=1)


In [3]:
numerical

,Unnamed: 0,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,total claim amount,month
0,0,4809.216960,48029,61,7.0,52,0.0,9,292.800000,2
1,1,2228.525238,0,64,3.0,26,0.0,1,744.924331,1
2,2,14947.917300,22139,100,34.0,31,0.0,2,480.000000,2
3,3,22332.439460,49078,97,10.0,3,0.0,2,484.013411,1
4,4,5035.035257,37405,63,8.0,99,3.0,4,287.556107,2
...,...,...,...,...,...,...,...,...,...,...
9129,9129,2771.663013,59855,74,30.0,82,4.0,1,355.200000,1
9130,9130,19228.463620,0,187,14.0,32,0.0,2,897.600000,2
9131,9131,5259.444853,61146,65,7.0,68,0.0,6,273.018929,1
9132,9132,23893.304100,39837,201,11.0,63,0.0,2,381.306996,2


In [4]:
numerical_x

,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception
0,-0.334914,0.218967,-1.546873,-0.819065,0.214078
1,-1.864614,-0.295792,-1.260678,-1.462015,-0.743570
2,1.293258,-1.333005,0.628149,1.623408,-0.543420
3,1.732720,0.265959,0.534556,-0.441404,-1.919838
4,-0.255895,-0.309653,-1.352362,-0.686531,1.645332
...,...,...,...,...,...
9129,-1.388766,0.707771,-0.507518,1.340769,1.156594
9130,1.576001,-0.295792,1.862420,-0.007926,-0.504540
9131,-0.181957,0.756361,-1.172446,-0.819065,0.731847
9132,1.800855,-0.179044,1.945201,-0.326883,0.574219


In [5]:
categorical

,Unnamed: 0,customer,state,response,coverage,education,effective to date,employmentstatus,gender,location code,marital status,policy type,policy,renew offer type,sales channel,vehicle class,vehicle size
0,0,DK49336,Arizona,No,Basic,College,2011-02-18,Employed,M,Suburban,Married,Corporate Auto,Corporate L3,Offer3,Agent,Four-Door Car,Medsize
1,1,KX64629,California,No,Basic,College,2011-01-18,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer4,Call Center,Four-Door Car,Medsize
2,2,LZ68649,Washington,No,Basic,Bachelor,2011-02-10,Employed,M,Suburban,Single,Personal Auto,Personal L3,Offer3,Call Center,SUV,Medsize
3,3,XL78013,Oregon,Yes,Extended,College,2011-01-11,Employed,M,Suburban,Single,Corporate Auto,Corporate L3,Offer2,Branch,Four-Door Car,Medsize
4,4,IW72280,California,No,Basic,Doctor,2011-02-14,Employed,F,Urban,Married,Corporate Auto,Corporate L2,Offer2,Branch,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,9129,SU71163,Arizona,No,Basic,College,2011-01-07,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer2,Branch,Two-Door Car,Medsize
9130,9130,QI63521,Nevada,No,Basic,High School or Below,2011-02-24,Unemployed,M,Suburban,Single,Personal Auto,Personal L2,Offer1,Branch,Luxury SUV,Medsize
9131,9131,KX53892,Oregon,No,Basic,College,2011-01-06,Employed,F,Urban,Married,Personal Auto,Personal L3,Offer2,Branch,Four-Door Car,Medsize
9132,9132,TL39050,Arizona,No,Extended,Bachelor,2011-02-06,Employed,F,Rural,Married,Corporate Auto,Corporate L3,Offer1,Web,Luxury SUV,Medsize


In [6]:
categorical_x

,Arizona,California,Nevada,Oregon,Washington,No,Yes,Basic,Extended,Premium,...,Web,Four-Door Car,Luxury Car,Luxury SUV,SUV,Sports Car,Two-Door Car,Large,Medsize,Small
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9130,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9131,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9132,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
numerical['number of open complaints'].value_counts()

0.0    7252
1.0    1011
2.0     374
3.0     292
4.0     149
5.0      56
Name: number of open complaints, dtype: int64

In [8]:
numerical['number of policies'].value_counts()

1    3251
2    2294
3    1168
7     433
9     416
4     409
5     407
8     384
6     372
Name: number of policies, dtype: int64

In [9]:
numerical_extra = pd.DataFrame()
numerical_extra['number of open complaints'] = numerical['number of open complaints']
numerical_extra['number of policies'] = numerical['number of policies']
numerical_extra

,number of open complaints,number of policies
0,0.0,9
1,0.0,1
2,0.0,2
3,0.0,2
4,3.0,4
...,...,...
9129,4.0,1
9130,0.0,2
9131,0.0,6
9132,0.0,2


In [10]:
# DOES NOT IMPROVE MODEL

# one_hot_names = []
# for col in numerical_extra.columns:
#     col_uniques = sorted(numerical_extra[col].unique())
#     for unique in col_uniques:
#         one_hot_names.append(col+'_'+str(unique))

# numerical_extra_x = pd.DataFrame(OneHotEncoder().fit(pd.DataFrame(numerical_extra))
#                              .transform(pd.DataFrame(numerical_extra)).toarray())
# numerical_extra_x.columns = one_hot_names

In [11]:
numerical_x['0 Open Complaints'] = np.where(numerical['number of open complaints'] > 0, 999, numerical['number of open complaints'])
numerical_x['0 Open Complaints'] = np.where(numerical_x['0 Open Complaints'] == 0, 1, numerical_x['0 Open Complaints'])
numerical_x['0 Open Complaints'] = np.where(numerical_x['0 Open Complaints'] == 999, 0, numerical_x['0 Open Complaints'])
numerical_x['1 Open Complaints'] = np.where(numerical['number of open complaints'] > 1, 0, numerical['number of open complaints'])
numerical_x['2+ Open Complaints'] = np.where(numerical['number of open complaints'] <= 1, 0, numerical['number of open complaints'])
numerical_x['2+ Open Complaints'] = np.where(numerical_x['2+ Open Complaints'] >= 2, 1, numerical_x['2+ Open Complaints'])

numerical_x

,customer lifetime value,income,monthly premium auto,months since last claim,months since policy inception,0 Open Complaints,1 Open Complaints,2+ Open Complaints
0,-0.334914,0.218967,-1.546873,-0.819065,0.214078,1.0,0.0,0.0
1,-1.864614,-0.295792,-1.260678,-1.462015,-0.743570,1.0,0.0,0.0
2,1.293258,-1.333005,0.628149,1.623408,-0.543420,1.0,0.0,0.0
3,1.732720,0.265959,0.534556,-0.441404,-1.919838,1.0,0.0,0.0
4,-0.255895,-0.309653,-1.352362,-0.686531,1.645332,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
9129,-1.388766,0.707771,-0.507518,1.340769,1.156594,0.0,0.0,1.0
9130,1.576001,-0.295792,1.862420,-0.007926,-0.504540,1.0,0.0,0.0
9131,-0.181957,0.756361,-1.172446,-0.819065,0.731847,1.0,0.0,0.0
9132,1.800855,-0.179044,1.945201,-0.326883,0.574219,1.0,0.0,0.0


In [12]:
categorical['effective to date'] = pd.to_datetime(categorical['effective to date'], errors='coerce')
categorical_extra = pd.DataFrame()
categorical_extra['month'] = categorical['effective to date'].dt.month
categorical_extra.value_counts()
categorical_x['January'] = np.where(categorical_extra['month'] > 1, 0, categorical_extra['month'])
categorical_x['January'].value_counts()

1    4898
0    4236
Name: January, dtype: int64

In [13]:
x = pd.DataFrame(np.concatenate((numerical_x,categorical_x),axis=1))
y = numerical['total claim amount']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)
model = sm.OLS(y_train,x_train).fit()
predictions = model.predict(x_test)
r2 = r2_score(y_test,predictions)
r2

0.777164485945901

# Bonus: Build a function, from round 2 and round 7, to clean and process the data

In [15]:
# Will be in round 8